In [2]:
from fastai.vision import *
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'fastai'

In [ ]:
download_images(urls="brand/ds3.txt", dest="brand/train/ds3", max_pics=200)

In [ ]:
download_images(urls="brand/ds5.txt", dest="brand/train/ds5", max_pics=200)

In [ ]:
classes = ['ds3', 'ds5']

In [ ]:
for c in classes:
    verify_images("brand/train"+c, delete=True)

In [ ]:
data = ImageDataBunch.from_folder(path="brand", train="train", test="test", valid_pct=0.25, ds_tfms=get_transforms(), size=224, bs=16).normalize(imagenet_stats)

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=accuracy)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot(suggestion=True)

In [ ]:
lr = learn.recorder.min_grad_lr
learn.fit_one_cycle(4, slice(lr))

In [ ]:
preds = learn.get_preds(ds_type=DatasetType.Test)

In [ ]:
preds